# Simulation of an Electric System based on Thermoelectrics Plants

In [64]:
from src.map import Map2D, GraphMap
from src.utils.gaussianmixture import DailyElectricityConsumptionBimodal
from src.circuits import Circuit, Block
from src.thermoelectrics import Thermoelectric
import pandas as pd
from numpy.random import default_rng
import plotly.express as px

from src.simulation_constants import (
    NO_CIRCUITS,
    NO_THERMOELECTRICS,
    MIN_CITIZEN,
    MAX_CITIZEN,
    MAX_DEVIATION_CITIZEN_IN_BLOCK,
    DEMAND_PER_PERSON,
    DEMAND_INDUSTRIALIZATION,
    VARIABILITY_DEMAND_PER_PERSON,
    VARIABILITY_DEMAND_PER_INDUSTRIALIZATION,
    PEAK_CONSUMPTION_MORNING,
    PEAK_CONSUMPTION_EVENING,
    MAX_DEVIATION_MORNING,
    MAX_DEVIATION_EVENING,
    WEIGHT_MORNING,
    WEIGHT_EVENING,
    RANDOM_SEED,
    MIN_BLOCKS_PER_CIRCUIT,
    MAX_BLOCKS_PER_CIRCUIT,
    IMPORTANCE_ALPHA,
)


## Initialize bases 

In [65]:
map_2d = Map2D(
    no_circuits=NO_CIRCUITS,
    no_thermoelectrics=NO_THERMOELECTRICS,
)


map_2d.visualize()

In [66]:
graphMap = GraphMap(
    thermoelectric_labels=[f"Th{i}" for i in range(NO_THERMOELECTRICS)],
    circuits_labels=[f"C{i}" for i in range(NO_CIRCUITS)],
    towers_labels=[f"Tw{i}" for i in range(len(map_2d.towers_positions))],
    thermoelectrics_positions=map_2d.thermoelectrics_positions,
    circuits_positions=map_2d.circuits_positions,
    towers_positions=map_2d.towers_positions,
)

graphMap.visualize()

In [67]:
distance_cost_template = graphMap.thermoelectric_generation_cost

# Generate circuits for map
rng = default_rng(RANDOM_SEED)

electric_system_circuits_list: list[Circuit] = []
for i in range(NO_CIRCUITS):
    citizen_count = rng.integers(MIN_CITIZEN, MAX_CITIZEN)
    citizen_range = (
        max(citizen_count - MAX_DEVIATION_CITIZEN_IN_BLOCK, 0),
        min(citizen_count + MAX_DEVIATION_CITIZEN_IN_BLOCK, MAX_CITIZEN),
    )

    industrialization = (
        rng.integers(0, DEMAND_INDUSTRIALIZATION) / DEMAND_INDUSTRIALIZATION
    )

    bimodal_consumption = DailyElectricityConsumptionBimodal(
        base_consumption=DEMAND_PER_PERSON * citizen_count
        + DEMAND_INDUSTRIALIZATION * industrialization,
        base_variability=VARIABILITY_DEMAND_PER_PERSON * citizen_count
        + VARIABILITY_DEMAND_PER_INDUSTRIALIZATION * industrialization,
        mean_morning=PEAK_CONSUMPTION_MORNING,
        mean_evening=PEAK_CONSUMPTION_EVENING,
        std_morning=rng.uniform(1.0, MAX_DEVIATION_MORNING),
        std_evening=rng.uniform(1.0, MAX_DEVIATION_EVENING),
        weight_morning=WEIGHT_MORNING,
        weight_evening=WEIGHT_EVENING,
    )

    electric_system_circuits_list.append(
        Circuit(
            graphMap.circuits_nodes[i].id,
            gaussian_mixture=bimodal_consumption,
            blocks_range=(MIN_BLOCKS_PER_CIRCUIT, MAX_BLOCKS_PER_CIRCUIT),
            citizens_range=citizen_range,
            industrialization=industrialization,
        )
    )

# Generate Thermoelectric generation based in the nearest circuits
mapper_circuit_with_thermoelectric = {}

for c in electric_system_circuits_list:

    filtered = [f for f in graphMap.thermoelectric_generation_cost if f[1] == c.id]
    filtered = sorted(filtered, key=lambda x: x[2])
    mapper_circuit_with_thermoelectric[c.id] = filtered[0][0]


import plotly.graph_objects as go

# Create edges for the plotly graph
edges = []
edges_labels = []
for circuit, thermoelectric in mapper_circuit_with_thermoelectric.items():
    
    circuitPos = [c.position for c in graphMap.circuits_nodes if c.id == circuit][0]
    thermoelectricPos = [
        t.position for t in graphMap.thermoelectrics_nodes if t.id == thermoelectric
    ][0]

    edges.append((circuitPos, thermoelectricPos))
    edges_labels.append(f"{circuit} -> {thermoelectric}")


# Create a plotly graph
fig = go.Figure()

# Add edges to the plotly graph
for i, edge in enumerate(edges):
    circuit, thermoelectric = edge
    fig.add_trace(
                go.Scatter(
                    x=[circuit[0], thermoelectric[0]],
                    y=[circuit[1], thermoelectric[1]],
                    mode="lines",
                    line=dict(color="black"),
                    name="Wire Connection",
                    showlegend=False,
                    hoverinfo="text",
                    text=edges_labels[i],
                    hoverlabel=dict(
                        bgcolor="white",
                        font_size=16,
                        font_family="Rockwell"
                    ),
                )
            )

# Add nodes to the plotly graph
all_nodes = graphMap.circuits_nodes + graphMap.thermoelectrics_nodes
for i,node in enumerate(all_nodes):
    fig.add_trace(
        go.Scatter(
            x=[node.position[0]],
            y=[node.position[1]],
            mode="markers+text",
            marker=dict(size=10, color="blue" if i >= len(graphMap.circuits_nodes) else "red"),
            text=[node.id],
            textposition="top center",
            hoverinfo="text",
        )
    )

fig.update_layout(showlegend=False, title="Circuits and Thermoelectrics", height=800, plot_bgcolor="lightgreen",)

fig.show()

In [68]:
# Generate the distance matrix


def distance_template_to_distance_matrix(
    template: list[tuple[str, str, float, list[str]]],
    thermoelectrics: list[str],
    circuits: list[str],
):
    matrix = [[-1 for _ in range(len(circuits))] for _ in range(len(thermoelectrics))]

    c_map = {}
    t_map = {}

    for i, t in enumerate(thermoelectrics):
        t_map[t] = i

    for i, c in enumerate(circuits):
        c_map[c] = i

    max_cost = 1
    for t, c, cost, _ in template:
        matrix[t_map[t]][c_map[c]] = cost
        max_cost = max(max_cost, cost)

    for t, c, _, _ in template:
        matrix[t_map[t]][c_map[c]] /= max_cost

    return matrix


matrix = distance_template_to_distance_matrix(
    distance_cost_template,
    [t.id for t in graphMap.thermoelectrics_nodes],
    [c.id for c in graphMap.circuits_nodes],
)


distance_matrix_df = pd.DataFrame(
    matrix,
    index=[t.id for t in graphMap.thermoelectrics_nodes],
    columns=[c.id for c in graphMap.circuits_nodes],
)

distance_matrix_df

,C0,C1,C2,C3,C4,C5,C6,C7,C8,C9,...,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49
Th0,0.454125,0.317891,0.411494,0.181482,0.346023,0.023278,0.464180,0.561002,0.274103,0.094344,...,0.386844,0.260235,0.483469,0.043787,0.351375,0.004352,0.271144,0.246672,0.500047,0.131370
Th1,0.660974,0.135453,0.618344,0.388331,0.163584,0.230128,0.281741,0.767851,0.091664,0.301193,...,0.204406,0.467085,0.690319,0.163415,0.168936,0.211202,0.477994,0.064233,0.706897,0.084630
Th2,0.053426,0.752901,0.038574,0.289308,0.781033,0.458288,0.899189,0.160303,0.709112,0.342444,...,0.821854,0.198441,0.082770,0.478796,0.786385,0.439362,0.209351,0.681682,0.099348,0.566380
Th3,0.081966,0.828510,0.114183,0.364918,0.856642,0.533897,0.974799,0.062425,0.784722,0.418053,...,0.897463,0.274051,0.057524,0.554406,0.861994,0.514971,0.284960,0.757291,0.015564,0.641989
Th4,0.791923,0.066086,0.749293,0.519280,0.024792,0.361077,0.142948,0.898801,0.072207,0.432143,...,0.077411,0.598034,0.821268,0.294364,0.030144,0.342151,0.608943,0.123284,0.837846,0.215579
Th5,0.893123,0.167285,0.850492,0.620479,0.115713,0.462276,0.028635,1.000000,0.173407,0.533342,...,0.178610,0.699233,0.922467,0.395563,0.106135,0.443350,0.710142,0.224483,0.939045,0.316778
Th6,0.375271,0.397867,0.332640,0.102627,0.425998,0.103254,0.544155,0.482148,0.354078,0.015490,...,0.466820,0.181381,0.404615,0.123762,0.431350,0.084328,0.192290,0.326647,0.421193,0.211345
Th7,0.290533,0.481483,0.247903,0.017890,0.509615,0.186870,0.627771,0.397411,0.437694,0.071026,...,0.550436,0.096644,0.319878,0.207378,0.514967,0.167944,0.107553,0.410263,0.336456,0.294962


In [69]:
# Generate thermoelectrics
electric_system_thermoelectrics_list: list[Thermoelectric] = []

for i, t in enumerate(graphMap.thermoelectrics_nodes):
    generated_thermoelectric_min_cost = 0

    for j, c in enumerate(graphMap.circuits_nodes):
        if mapper_circuit_with_thermoelectric[c.id] == t.id:
            generated_thermoelectric_min_cost += (
                electric_system_circuits_list[j].mock_electric_consume
                + electric_system_circuits_list[j].mock_electric_consume
                * matrix[i][j]
                * 24
            )

    electric_system_thermoelectrics_list.append(
        Thermoelectric(
            id=t.id,
            parts=[],
            total_capacity=generated_thermoelectric_min_cost + 200,
        )
    )
    thermoelectric_data = {
        "ID": [t.id for t in electric_system_thermoelectrics_list],
        "Total Capacity": [
            t.total_capacity for t in electric_system_thermoelectrics_list
        ],
    }

    thermoelectric_df = pd.DataFrame(thermoelectric_data)


sorted_electric_system_circuit_list = sorted(
    electric_system_circuits_list,
    key=lambda x: list(
        filter(
            lambda y: mapper_circuit_with_thermoelectric[x.id] == y.id,
            graphMap.thermoelectrics_nodes,
        ))[0].id)

colors = [
    list(
        filter(
            lambda x, y=circuit: mapper_circuit_with_thermoelectric[y.id] == x.id,
            graphMap.thermoelectrics_nodes,
        )
    )[0].id
    for circuit in sorted_electric_system_circuit_list
]

print(colors)

print([t.id for t in electric_system_thermoelectrics_list])

fig = px.bar(
    thermoelectric_df,
    x="ID",
    y="Total Capacity",
    title="Total Capacity of Thermoelectrics",
    height=800,
    color=[t.id for t in electric_system_thermoelectrics_list],
    color_discrete_sequence=px.colors.qualitative.Plotly,
    labels={"color": "Thermoelectric ID"},
)

fig.show()

['Th0', 'Th0', 'Th0', 'Th0', 'Th0', 'Th0', 'Th0', 'Th0', 'Th0', 'Th0', 'Th0', 'Th0', 'Th1', 'Th1', 'Th1', 'Th1', 'Th1', 'Th1', 'Th1', 'Th2', 'Th2', 'Th2', 'Th2', 'Th2', 'Th2', 'Th2', 'Th2', 'Th3', 'Th3', 'Th3', 'Th3', 'Th3', 'Th3', 'Th4', 'Th4', 'Th4', 'Th4', 'Th4', 'Th4', 'Th4', 'Th5', 'Th5', 'Th5', 'Th6', 'Th7', 'Th7', 'Th7', 'Th7', 'Th7', 'Th7']
['Th0', 'Th1', 'Th2', 'Th3', 'Th4', 'Th5', 'Th6', 'Th7']


In [70]:

mock_electric_consume_data = {
    "Circuit ID": [c.id for c in sorted_electric_system_circuit_list],
    "Mock Electric Consume": [c.mock_electric_consume for c in sorted_electric_system_circuit_list],
}

mock_electric_consume_df = pd.DataFrame(mock_electric_consume_data)

fig = px.bar(
    mock_electric_consume_df,
    x="Circuit ID",
    y="Mock Electric Consume",
    title="Electric Consume per Circuit [24 hours]",
    height=800,
    color=colors
)
fig.show()

In [71]:
max_population_of_circuits = -1
max_population_of_block = -1

for circuit in electric_system_circuits_list:
    max_population_of_circuits = max(
        circuit.get_all_block_population(), max_population_of_circuits
    )
    for block in circuit.blocks:
        max_population_of_block = max(block.citizens.amount, max_population_of_block)

auxiliary_data_max_population_of_circuits = max_population_of_circuits
auxiliary_data_max_population_of_block = max_population_of_block


def get_circuit_importance(circuit: Circuit) -> float:
    return (
        circuit.get_all_block_population() / auxiliary_data_max_population_of_circuits
    ) * IMPORTANCE_ALPHA + circuit.industrialization * (1 - IMPORTANCE_ALPHA)


def get_block_importance(block: Block) -> float:
    return (
        block.citizens.amount / auxiliary_data_max_population_of_circuits
    ) * IMPORTANCE_ALPHA + block.industrialization * (1 - IMPORTANCE_ALPHA)


def set_importance(ci: list[Circuit]):
    for circuit in ci:
        for block in circuit.blocks:
            block.importance = get_block_importance(block)
        circuit.importance = get_circuit_importance(circuit)

set_importance(electric_system_circuits_list)

max_population_data = {
    "Max Population of Circuits": [auxiliary_data_max_population_of_circuits],
    "Max Population of Blocks": [auxiliary_data_max_population_of_block],
}

max_population_df = pd.DataFrame(max_population_data)

max_population_df

,Max Population of Circuits,Max Population of Blocks
0,19483,4917


In [72]:
# All data of one block of a circuit (importance, citizens, industrialization, demand per hour)

circuit_stats = electric_system_circuits_list[0]

blocks = circuit_stats.blocks

citizens = []
importance = []
demand_per_hour = []

for block in blocks:
    citizens.append(block.citizens.amount)
    importance.append(block.importance)
    demand_per_hour.append(sum(block.predicted_demand_per_hour))

block_stats_df = pd.DataFrame(
    {
        "Citizens": citizens,
        "Importance": importance,
        "Demand per Hour": demand_per_hour,
    }
)

fig_citizens = px.bar(
    block_stats_df,
    x=block_stats_df.index,
    y="Citizens",
    title=f"Block Citizens in Circuit {circuit_stats.id}",
    height=800,
)

fig_importance = px.bar(
    block_stats_df,
    x=block_stats_df.index,
    y="Importance",
    title=f"Block Importance in Circuit {circuit_stats.id}",
    height=800,
)

fig_demand_per_hour = px.bar(
    block_stats_df,
    x=block_stats_df.index,
    y="Demand per Hour",
    title=f"Block Demand per Hour in Circuit {circuit_stats.id}",
    height=800,
)


fig_citizens.show()
fig_importance.show()
fig_demand_per_hour.show()

In [73]:
# Compare data from circuits

citizens = []
importance = []
demand_per_hour = []

for circuit in sorted_electric_system_circuit_list:
    
    importance.append(circuit.importance)
    demand_per_hour.append(
        sum([sum(block.predicted_demand_per_hour) for block in circuit.blocks])
    )

# Separate for properties
citizens = [
    sum(block.citizens.amount for block in circuit.blocks)
    for circuit in electric_system_circuits_list
]

circuit_comparison_df = pd.DataFrame(
    {   "Circuit ID": [c.id for c in sorted_electric_system_circuit_list], 
        "Citizens": citizens,
        "Importance": importance,
        "Demand per Hour": demand_per_hour,
    }
)

fig_citizens = px.bar(
    circuit_comparison_df,
    x="Circuit ID",
    y="Citizens",
    title="Total Citizens per Circuit",
    height=800,
    color=colors,
    color_discrete_sequence=px.colors.qualitative.Plotly,
    labels={"color": "Thermoelectric ID"},
)

fig_importance = px.bar(
    circuit_comparison_df,
    x="Circuit ID",
    y="Importance",
    title="Importance per Circuit",
    height=800,
    color=colors,
    color_discrete_sequence=px.colors.qualitative.Plotly,
    labels={"color": "Thermoelectric ID"},
)

fig_demand_per_hour = px.bar(
    circuit_comparison_df,
    x="Circuit ID",
    y="Demand per Hour",
    title="Total Demand per Circuit in a Day",
    height=800,
    color=colors,
    color_discrete_sequence=px.colors.qualitative.Plotly,
    labels={"color": "Thermoelectric ID"},
)

fig_citizens.show()
fig_importance.show()
fig_demand_per_hour.show()

In [74]:
circuit_id = "C0"
circuit = next(c for c in electric_system_circuits_list if c.id == circuit_id)

block_demand_data = {"Block ID": [], "Hour": [], "Demand": []}

for i, block in enumerate(circuit.blocks):
    for hour, demand in enumerate(block.predicted_demand_per_hour):
        block_demand_data["Block ID"].append(f"Block {i + 1}")
        block_demand_data["Hour"].append(hour)
        block_demand_data["Demand"].append(demand)

block_demand_df = pd.DataFrame(block_demand_data)

fig_block_demand = px.line(
    block_demand_df,
    x="Hour",
    y="Demand",
    color="Block ID",
    title=f"Hourly Demand per Block in Circuit {circuit_id}",
    labels={"Demand": "Demand (kWh)", "Hour": "Hour"},
)

fig_block_demand.show()

circuit_demand_data = {
    "Hour": list(range(24)),
    "Demand": [
        sum(block.predicted_demand_per_hour[hour] for block in circuit.blocks)
        for hour in range(24)
    ],
}

circuit_demand_df = pd.DataFrame(circuit_demand_data)

fig_circuit_demand = px.line(
    circuit_demand_df,
    x="Hour",
    y="Demand",
    title=f"Hourly Demand of Circuit {circuit_id}",
    labels={"Demand": "Demand (kWh)", "Hour": "Hour"},
)

fig_circuit_demand.show()